___
# COMP1804 Lab 3 - Data Pre-processing



**Learning Objectives:**
 *  Understand the stages of data pre-processing.
 *  Use Python for pre-processing. 
 *  Practice the different stages of data pre-processing in Python.
___

## 1. Loading the Dataset


For this Lab, I have used a subset of the Loan Prediction dataset. You can download the training and testing dataset from here: [Download Data](https://drive.google.com/drive/folders/10beInU2j3J9tF_dpbt50_36oxoa9jJQx?usp=sharing)

Note : Testing data that you are provided is the subset of the training data from Loan Prediction problem.

 

Now, lets get started by importing important packages and the dataset.

**1.1 Import the necessary Python modules**

In [1]:
# Load python modules

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import model_selection


**1.2 Load Dataset **

Note: Download the csv files from the URL to your local drive and load from there as shown in the code below.

We are using pandas to load the data. We will also use pandas next to explore the data both with descriptive statistics and data visualization.


In [ ]:
# Load dataset from local drive (for colab notebook)
from google.colab import files
import io

uploaded = files.upload()    # Will prompt you to select file
train_dataset = pd.read_csv(io.BytesIO(uploaded['X_train.csv']))

uploaded = files.upload()    # Will prompt you to select file
train_labels = pd.read_csv(io.BytesIO(uploaded['Y_train.csv']))

uploaded = files.upload()    # Will prompt you to select file
test_dataset = pd.read_csv(io.BytesIO(uploaded['X_test.csv']))

uploaded = files.upload()    # Will prompt you to select file
test_labels = pd.read_csv(io.BytesIO(uploaded['Y_test.csv']))




**1.2.1 Inspect Dataset **

**1.2.1.1 Dimensions of Dataset **


In [ ]:
# shape
print(train_dataset.shape)
print(train_labels.shape)

In [ ]:
print(test_dataset.shape)
print(test_labels.shape)

In [ ]:
# list of column titles 
print(train_dataset.columns)
print(train_labels.columns)

In [ ]:
# list of column (field) data types
print(train_dataset.dtypes)
print(train_labels.dtypes)

**1.2.1.2 Take a peek at the Dataset **

Note that "NaN" means "Not a Number". It is not the same as 0. Python replaces empty fields with "NaN".


In [ ]:
# head
train_dataset.head(20)

##2. Managing Missing Data

Sometimes you may find some data are missing in the dataset. If the missing values are not handled properly inaccurate inference about the data may result. Due to improper handling, the result obtained will differ from ones where the missing values are present. 

Since missing values can tangibly reduce prediction accuracy, this step needs to be a priority. In terms of machine learning, assumed or approximated values are “more appropriate” for an algorithm than just missing ones.  Even if you don’t know the exact value, methods exist to better “assume” which value is missing or bypass the issue. So how to сlean the data here? Choosing the right approach also heavily depends on data and the domain you have:
* Substitute missing values with dummy values, e.g. n/a for categorical or 0 for numerical values.
* Substitute the missing numerical values with mean figures.
* For categorical values, you can also use the most frequent items to fill in.

### 2.1 Removing Missing Data

So how can we handle missing data. One obvious idea is to remove the lines (observations recorded in the rows) where there is some missing data. That is ok if you’ve got large datasets. The dataset we have here is considerably small, so removing data will have crucial impact. We will therefore need to look for alternative methods to deal with the missing data. Removing removing rows can be quite dangerous as you will be deleting crucial information.

The decission to remove data will depend on the size of dataset and the problem domain (type of data collected).

Assuming it was appropriate to remove observation rows, the following code will help. 
Given the original dataset in figure 2, we could remove rows 4 and 6 as they containg missing data.



**2.1.1 Remove all rows that contain missing data**


In [ ]:
# remove all rows with missing data
print(train_dataset.dropna())
train_dataset.head(20)



**2.1.2 Remove specific rows**

Drop specific rows by passing index labels to the drop method.

In [ ]:
# remove selected rows

print(train_dataset.drop([0]))
train_dataset.head(20)

**2.1.3 Remove specific columns**

Usually you would drop particular columns especially if all or most of its values are missing. The drop method can also be used here with parameters to define the column title and axis=1 to denote that we want to drop a column.


In [ ]:
# remove selected column

print(train_dataset.drop("Gender", axis=1))

In [ ]:
# remove multiple selected columns

print(train_dataset.drop(["Gender", "Married"], axis=1))

** 2.1.4 Remove all rows where data satisfies a condition in a particular column**

Drop all rows where Gender is Female (and NaN). So filter the dataframe where Gender is Male.


In [ ]:
# remove all rows where Gender is Female (and NaN).

print(train_dataset[train_dataset["Gender"] == "Male"])

** 2.1.4 Remove all rows where data satisfies a condition in a particular column**

Drop all rows where Gender is not Male. So keep rows where Gender is Female or NaN.

In [ ]:
# remove all rows where Gender is Female (and NaN).

print(train_dataset[train_dataset["Gender"] != "Male"])

**2.1.5 Remove all rows where data is missing in a particular column**

Remove all rows where a particular column has a missing value. This will result in a dataset that has no missing values in that particular column.


In [ ]:
# remove all rows where ApplicantIncome is missing.
# 1. check to see if there are missing data in the ApplicantIncome column
print(train_dataset["ApplicantIncome"].isnull().sum())

print(train_dataset.head(20))


# 2. drop all rows where a value for ApplicantIncome is missing
print(train_dataset[train_dataset["ApplicantIncome"].notnull()])

# the first row contains a null and is removed



### 2.2 Filling in Missing Data

As alternative to removing rows with missing data, you can estimate plausible values for the missing data instead. For example, replace the missing data within a column with a value equivalent to the mean of all the values in that column. That can also be applied to every feature where there is missing data.


We will use the scikit-learn library. The impute class allows us to manipulate the missing data.


At first we will create an object of the imputer class. 
The class has an argument called 'strategy' which shows what method to use to generate a value replacement for the missing value. 'mean' is the default value for strategy.

If “mean”, then replace missing values using the mean along each column. Can only be used with numeric data.

If “median”, then replace missing values using the median along each column. Can only be used with numeric data.

If “most_frequent”, then replace missing using the most frequent value along each column. Can be used with strings or numeric data. If there is more than one such value, only the smallest is returned.

If “constant”, then replace missing values with fill_value. Can be used with strings or numeric data.

In [ ]:
print(train_dataset["Gender"].isnull().sum())

print(train_dataset["ApplicantIncome"].isnull().sum())



In [ ]:
# handling missing data
from sklearn.impute import SimpleImputer 

train_dataset_no_nans =  train_dataset.copy()

# 1. Imputer
imptr = SimpleImputer(missing_values = np.nan, strategy = 'mean')  


# 2. Fit the imputer object to the feature matrix (only for numeric features)
imptr = imptr.fit(train_dataset_no_nans[['ApplicantIncome', 'CoapplicantIncome',
                'LoanAmount', 'Loan_Amount_Term', 'Credit_History']])

# 3. Call Transform to replace missing data in train_dataset (on specific columns) by the mean of the column to which that missing data belongs to
train_dataset_no_nans[['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']] = \
imptr.transform(train_dataset_no_nans[['ApplicantIncome', 'CoapplicantIncome','LoanAmount', 'Loan_Amount_Term', 'Credit_History']]) 

train_dataset_no_nans



In [320]:
# 1. Imputer
imptr = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')  

# 2. Fit the imputer object to the feature matrix (only for numeric features)
imptr = imptr.fit(train_dataset[['Gender']])

# 3. Call Transform to replace missing data in train_dataset (on specific columns) by the mean of the column to which that missing data belongs to
train_dataset_no_nans[['Gender']] = imptr.transform(train_dataset[['Gender']]) 



##############################################################################
##############################################################################
##############################################################################
##############################################################################




##  Exercise no 1

Fill in the missing values for the categorical features









In [ ]:
### insert code here; work on the: train_dataset_no_nans

##############################################################################
##############################################################################
##############################################################################
##############################################################################




##3.  Categorical Data

Data Preprocessing in machine learning requires values of the data in numerical form. Therefore text values in the columns of datasets must be converted into numerical form. 

###3.1 Converting categorical to numerical values

Given the original dataset, it is clear we have a few categorical features. All these need to be encoded. The [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) class is used to transform the categorical or string values to numerical ones (between 0 and n_classes-1).



In [ ]:
dummy = train_dataset_no_nans.copy()

# encode categorical data for the 'Gender' column
from sklearn.preprocessing import LabelEncoder
lblEncoder_X = LabelEncoder()                 # create an object of the LabelEncoder class
dummy['Gender'] = lblEncoder_X.fit_transform(dummy['Gender']) # apply LblEncoder object to our categorical variables (columns - 'Gender') using the fit_transform method. This returns the column encoded.

print(dummy)



Now let us try a different encoding strategy, the one-hot encoding.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

oneHotEncoder = OneHotEncoder()                                      # create OneHotEncoder object 
aaa = oneHotEncoder.fit(train_dataset_no_nans[['Gender']])           # fit the OneHotEncoder object to feature Gender
aaa.categories_

dum = aaa.transform(train_dataset_no_nans[['Gender']]).toarray()               # dum is an array of shape (391,2) containing the one-hot encoding of the feature Gender of the dataframe train_dataset_no_nans
dum

train_dataset_no_nans['Female'] = dum[:,0]     # we add to the train_dataset_no_nans one feature column called Female and add there the corresponding encoded values
train_dataset_no_nans['Male'] = dum[:,1]       # we add to the train_dataset_no_nans one feature column called Male and add there the corresponding encoded values
train_dataset_no_nans= train_dataset_no_nans.drop(columns='Gender',axis=1)  # we delete the (former) column/feature Gender

train_dataset_no_nans

##############################################################################
##############################################################################
##############################################################################
##############################################################################





##  Exercise no 2

Perform label encoding and one-hot encoding for all categorical features.



In [ ]:
# place your code here

##  Exercise no 3

Perform label encoding for the labels (i.e., the train_labels **dataframe**)



In [ ]:
# place your code here

##############################################################################
##############################################################################
##############################################################################
##############################################################################





##4.  Compute Statistics and Check Imbalance

In [ ]:
import matplotlib.pyplot as plt

# compute histograms of some numeric features (you can see the ranges)
train_dataset_no_nans[train_dataset_no_nans.dtypes[(train_dataset_no_nans.dtypes=="float64") | (train_dataset_no_nans.dtypes=="int64")].index.values].hist(figsize=[15,15])  # the syymbol '|' means 'or'



In [ ]:
sampleClassBias = train_labels['Target'].value_counts()
print('Training Labels distribution:')
print(sampleClassBias)

sampleClassBias = test_labels['Target'].value_counts()
print('Test Labels distribution:')
print(sampleClassBias)

##5.  Feature Scaling

When the data is comprised of feature values with varying scales, many machine learning algorithms can benefit from rescaling the attributes to all have the same scale. 


###5.1 Implementing Feature Scaling

Feature scaling is a method used to scale the range of variables/values of features. 

>![feature scaling](https://drive.google.com/uc?id=1loaMbVo_7ZJo53Ogv7I0XMG_wSOOYBxI)

>  Figure 5: Feature Scaling methods
> Where X is the observation feature.

There are several ways of scaling the data. One way is called **Standardisation** which may be used. For every observation of the selected column, our program will apply the formula of standardisation and fit it to a scale. That is for each observation and each feature within the mean value is withdrawn (subtracted ) from all the values of the feature and divide by the standard deviation. 
The other common type of scaling is **normalisation** where the minimal value of all the feature values is subtracted from the observation feature X and divided by the difference between the max of the feature values and the min of the feature values.

**It is not essential to understand the math behind these methods, what is important is to remember that the variables are being put in the same range / same scale so that no variable is dominated by another.**


In [325]:
#normalisation
ddummy = train_dataset_no_nans.copy()


# Importing MinMaxScaler and initializing it
from sklearn.preprocessing import MinMaxScaler
min_max=MinMaxScaler()
# Normalising
train_dataset_no_nans[['ApplicantIncome', 'CoapplicantIncome','LoanAmount', 'Loan_Amount_Term', 'Credit_History']]=min_max.fit_transform(train_dataset_no_nans[['ApplicantIncome', 'CoapplicantIncome',
                'LoanAmount', 'Loan_Amount_Term', 'Credit_History']])

train_dataset_no_nans


Question: Do we need to apply feature scaling to the labels? The answer is no! The label is a categorical value that takes 2 values either no (0) or yes (1).  It is a classification problem.


In [326]:
# Standardisation
train_dataset_no_nans1 = ddummy.copy()

#Standardizing 
from sklearn.preprocessing import scale
train_dataset_no_nans1[['ApplicantIncome', 'CoapplicantIncome','LoanAmount', 'Loan_Amount_Term', 'Credit_History']]=scale(train_dataset_no_nans1[['ApplicantIncome', 'CoapplicantIncome',
                'LoanAmount', 'Loan_Amount_Term', 'Credit_History']])
train_dataset_no_nans1

##############################################################################
##############################################################################
##############################################################################
##############################################################################





##  Exercise no 4

Perform normalisation on the other features too.



In [ ]:
# place code here

##  Exercise no 5

Perform standardisation on the other features too.



In [ ]:
# place code here

##############################################################################
##############################################################################
##############################################################################
##############################################################################





##7. Final

The steps below can be used as a base for ML projects with small variation.

##  Exercise no 6

Perform all the previous data pre-processing steps to the test data and labels.

In [ ]:
# place code here

Next I provide a standard classifier so that you perform some tests

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


# we will use a classifier (we will skip details and what it does for now, so just use it as it is)
knn=KNeighborsClassifier(n_neighbors=5)
knn.fit(train_dataset_no_nans1, train_labels['Target'])  
## we train the classifier with the training data and labels: train_dataset_no_nans1 should be the training dataframe after:  
## i) filling in all missing values, ii) encoding all categorical features and (maybe) after feature scaling

# Checking the model's accuracy (performance); this should be performed on the test set and thus we use the test_dataset_no_nans1 (same pre-processing as before shoud have been performed) and the test labels (after encoding)
accuracy_score(test_labels['Target'],knn.predict(test_dataset_no_nans1]))

##  Final Exercise 

Test the performance of the classifier when:

- the label encoder has been used and no feature scaling has been performed
- the label encoder has been used and normalisation has been performed 
- the label encoder has been used and standardisation has been performed 
- the one-hot encoder has been used and no feature scaling has been performed
- the one-hot encoder has been used and normalisation has been performed 
- the one-hot encoder has been used and standardisation has been performed 

Maybe the performance can be further improved? Have a look at the features that you used and think whether data quality assessment can be performed.